In [1]:
from selenium import webdriver       # 파이썬으로 브라우저 컨트롤 모듈
from bs4 import BeautifulSoup as bs # 크롤링한 데이터 정제
import pandas as pd                  # 데이터 분석 모듈

In [2]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.starbucks.co.kr/store/store_map.do')

C:\Users\kyg83\AppData\Local\Temp/ipykernel_2680/2198966681.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [3]:
area_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
driver.find_element_by_css_selector(area_btn).click()

C:\Users\kyg83\AppData\Local\Temp/ipykernel_2680/2162603177.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(area_btn).click()


In [4]:
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()

C:\Users\kyg83\AppData\Local\Temp/ipykernel_2680/3501212899.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(seoul_btn).click()


In [5]:
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()

C:\Users\kyg83\AppData\Local\Temp/ipykernel_2680/2684697461.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(all_btn).click()


In [6]:
html = driver.page_source  # 전체 페이지 소스 읽어오기
soup = bs(html)             # bs으로 데이터 정제

starbucks_list = soup.select('ul.quickSearchResultBoxSidoGugun > li.quickResultLstCon')          # 서울 전체 스타벅스 목록만 읽어오기
starbucks_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [7]:
star = starbucks_list[0]
star

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [8]:
# name = star['data-name']   # 지점명
name = star.select('strong')[0].text.strip()
lat =  star['data-lat'].strip()  # 위도
lng = star['data-long'].strip()     # 경도
addr = str(star.select('p')[0]).split('<br/>')[0].split('>')[1]  # 주소
tel = str(star.select('p')[0]).split('<br/>')[1].split('<')[0]   # 전화번호


print(name)
print(lat)
print(lng)
print(addr)
print(tel)

역삼아레나빌딩
37.501087
127.043069
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [9]:
starbucks = []

for st in starbucks_list:
    name = st.select('strong')[0].text.strip()
    lat =  st['data-lat'].strip()
    lng = st['data-long'].strip()
    addr = str(st.select('p')[0]).split('<br/>')[0].split('>')[1]
    tel = str(st.select('p')[0]).split('<br/>')[1].split('<')[0]
    starbucks.append([name, lat, lng, addr, tel])
    
columns = ['매장명','위도','경도','주소','전화번호']
seoul_starbucks = pd.DataFrame(starbucks, columns = columns)

seoul_starbucks.to_excel('Starbucks.xlsx', index=False)

In [10]:
# seoul_starbucks 데이터 프레임에 '시군구' 이름의 열을 만들어서
# 주소에서 구만 추출하여 삽입하시오.
sgg_names = []

for addr in seoul_starbucks['주소']:
    sgg = addr.split()[1]
    sgg_names.append(sgg)
    
seoul_starbucks['시군구명'] = sgg_names

seoul_starbucks.head()

,매장명,위도,경도,주소,전화번호,시군구명
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구


In [11]:
starbucks_count = seoul_starbucks.pivot_table(
                                             index = '시군구명',
                                             values = '매장명',
                                             aggfunc = 'count')
starbucks_count

,매장명
시군구명,
강남구,88
강동구,16
강북구,6
강서구,22
관악구,11
광진구,18
구로구,12
금천구,11
노원구,13


In [12]:
# 스타벅스 지도 그리기
# !pip install folium
import folium

In [13]:
starbucks_map = folium.Map(location=[37.5665,126.9780], zoom_start=11, tiles = 'Stamen Terrain')
starbucks_map

In [14]:
# 37.501087 127.043069

folium.Marker(location=[37.501087, 127.043069]).add_to(starbucks_map)

starbucks_map

In [16]:
for i in range(len(seoul_starbucks)):
    folium.Marker(location=[seoul_starbucks['위도'][i],seoul_starbucks['경도'][i]]).add_to(starbucks_map)
    
starbucks_map

In [95]:
lota = []

for lat in seoul_starbucks['위도']:
    lota.append(lat)

lnga = []

for lng in seoul_starbucks['경도']:
    lnga.append(lng)
    
for i in range(len(seoul_starbucks)):
    folium.Marker(location=[lota[i], lnga[i]]).add_to(starbucks_map)
    

starbucks_map
